In [1]:
# !pip install biopython
!pip install --upgrade --no-cache-dir biopython


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install rdkit-pypi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 29.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-2.2.0+cu118.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-2.2.0+cu118.html
!pip install -q torch-geometric


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [18]:
import numpy as np
from collections import Counter
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from Bio.Align import substitution_matrices
from sklearn.cluster import KMeans
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
import itertools
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch_geometric.data import Batch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import torch



blosum62 = substitution_matrices.load('BLOSUM62')

In [5]:
#30
def apaac(sequence, lambda_value=10, weight=0.05):
    
    # Hydrophobicity (h1) and Hydrophilicity (h2)
    hydrophobicity = {
        'A': 0.62,  'C': 0.29,  'D': -0.90, 'E': -0.74, 'F': 1.19,
        'G': 0.48,  'H': -0.40, 'I': 1.38,  'K': -1.50, 'L': 1.06,
        'M': 0.64,  'N': -0.78, 'P': 0.12,  'Q': -0.85, 'R': -2.53,
        'S': -0.18, 'T': -0.05, 'V': 1.08,  'W': 0.81,  'Y': 0.26
    }
    
    hydrophilicity = {
        'A': -0.50, 'C': -1.00, 'D': 3.00,  'E': 3.00,  'F': -2.50,
        'G': 0.00,  'H': -0.50, 'I': -1.80, 'K': 3.00,  'L': -1.80,
        'M': -1.30, 'N': 0.20,  'P': 0.00,  'Q': 0.20,  'R': 3.00,
        'S': 0.30,  'T': -0.40, 'V': -1.50, 'W': -3.40, 'Y': -2.30
    }
    
    
    amino_acids = list(hydrophobicity.keys())
    sequence = sequence.upper()  
    
    # Compute standard amino acid composition (AAC)
    aac = np.array([sequence.count(aa) / len(sequence) for aa in amino_acids])

    # Compute sequence-order correlation factors
    lambda_correlation = []
    for i in range(1, lambda_value + 1):
        sum_corr = 0
        for j in range(len(sequence) - i):
            if sequence[j] in amino_acids and sequence[j + i] in amino_acids:
                h1_corr = (hydrophobicity[sequence[j]] - hydrophobicity[sequence[j + i]])**2
                h2_corr = (hydrophilicity[sequence[j]] - hydrophilicity[sequence[j + i]])**2
                sum_corr += (h1_corr + h2_corr) / 2  # Average correlation
        lambda_correlation.append((sum_corr+1e-7) / ((len(sequence) - i)+1e-7))

    lambda_correlation = np.array(lambda_correlation)

    # Normalize and combine features
    apaac_vector = np.concatenate((aac * (1 - weight * sum(lambda_correlation)), weight * lambda_correlation))
    
    return apaac_vector

In [5]:
# seq = "ACDEFGHIKLMNPQRSTVWYACDEFGHIKLMNPQRSTVWYX"
# apaac_vector = apaac(seq)
# print("APAAC Feature Vector:", apaac_vector)
# print("Feature Vector Length:", len(apaac_vector))


APAAC Feature Vector: [-0.06343104 -0.06343104 -0.06343104 -0.06343104 -0.06343104 -0.06343104
 -0.06343104 -0.06343104 -0.06343104 -0.06343104 -0.06343104 -0.06343104
 -0.06343104 -0.06343104 -0.06343104 -0.06343104 -0.06343104 -0.06343104
 -0.06343104 -0.06343104  0.20651625  0.24157526  0.20727237  0.33520682
  0.24398625  0.1566355   0.27018647  0.18447924  0.18053359  0.27394452]
Feature Vector Length: 30


In [6]:
#15
def ctdd(sequence):
    secondary_structure = {
    "Helix": set("EALMQKRH"),
    "Strand": set("VIYCWFT"),
    "Coil": set("GNPSD"),
    }

    ctdd_vector = []
    sequence_length=len(sequence)
    for class_name, amino_acids in secondary_structure.items():
        positions = [i for i, aa in enumerate(sequence) if aa in amino_acids]
        
        if not positions:  # If no amino acid of this class is found
            ctdd_vector.extend([0, 0, 0, 0, 0])
            continue

        # Calculate the five key positions (first, 25%, 50%, 75%, last)
        first = positions[0] / sequence_length
        p25 = positions[int(len(positions) * 0.25)] / sequence_length
        p50 = positions[int(len(positions) * 0.50)] / sequence_length
        p75 = positions[int(len(positions) * 0.75)] / sequence_length
        last = positions[-1] / sequence_length

        ctdd_vector.extend([first, p25, p50, p75, last])

    return ctdd_vector



In [7]:
seq = "ACDEFGHIKLMNPQRSTVWYACDEFGHIKLMNPQRSTVWYX"
ctdd_vector = ctdd(seq)
print("CTDD Feature Vector:", ctdd_vector)
print("Feature Vector Length:", len(ctdd_vector))

CTDD Feature Vector: [0.0, 0.21951219512195122, 0.4878048780487805, 0.7073170731707317, 0.8292682926829268, 0.024390243902439025, 0.3902439024390244, 0.5121951219512195, 0.8780487804878049, 0.9512195121951219, 0.04878048780487805, 0.2682926829268293, 0.5365853658536586, 0.7560975609756098, 0.8536585365853658]
Feature Vector Length: 15


In [7]:
#343
def ctriad(sequence):
    #Grpups based on their dipoles and side-chain volumes
    amino_acid_groups = {
    'A': 1, 'G': 1, 'V': 1,  # Group 1
    'I': 2, 'L': 2, 'F': 2, 'P': 2,  # Group 2
    'Y': 3, 'M': 3, 'T': 3, 'S': 3,  # Group 3
    'H': 4, 'N': 4, 'Q': 4, 'W': 4,  # Group 4
    'R': 5, 'K': 5,  # Group 5
    'D': 6, 'E': 6,  # Group 6
    'C': 7   # Group 7
     }
    # Convert sequence to reduced alphabet (group numbers)
    reduced_seq = [amino_acid_groups[aa] - 1 for aa in sequence if aa in amino_acid_groups]
    # Extract triads
    triads = [tuple(reduced_seq[i:i+3]) for i in range(len(reduced_seq) - 2)]
   
    # Count occurrences of each triad
    triad_counts = Counter(triads)

    # Normalize counts
    total_triads = len(triads)
    triad_vector = np.zeros((7, 7, 7))  # 7^3 possible triads

    for triad, count in triad_counts.items():
        triad_vector[triad] = count / total_triads  # Normalize frequency

    return triad_vector.flatten()



In [9]:

seq = "ACDEFGHIKLMNPQRSTVWYACDEFGHIKLMNPQRSTVWYX"
ctriad_vector = ctriad(seq)

print("CTriad Feature Vector:", ctriad_vector)
print("Feature Vector Length:", len(ctriad_vector))

CTriad Feature Vector: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.05263158 0.05263158
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.05263158
 0.         0.         0.         0.         0.05263158 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.05263158 0.         0.         0.         0.         0.
 0.         0.         0.05263158 0.         0.         0.
 0.05263158 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.   

In [8]:
#5
def get_similarity(seq1, seq2):
    """Compute similarity score between two subsequences using BLOSUM62."""
    return sum(blosum62.get((a, b), blosum62.get((b, a), -4)) for a, b in zip(seq1, seq2))


def extract_subsequences(protein_seqs, k=5):
    """Extract fixed-length k-mers from a list of protein sequences."""
    all_subsequences = set()
    protein_subsequences = []  # Store k-mers for each protein separately

    for seq in tqdm(protein_seqs,desc='extract_subsequences'):
        kmers = [seq[i:i+k] for i in range(len(seq) - k + 1)]
        protein_subsequences.append(kmers)  # Store per protein
        all_subsequences.update(kmers)  # Collect all k-mers for clustering

    return all_subsequences, protein_subsequences

def compute_similarity_matrix(subsequences, n_jobs=-1):
    n = len(subsequences)
    similarity_matrix = np.zeros((n, n))

    def compute_row(i):
        row = np.zeros(n)
        for j in range(i, n):
            score = get_similarity(subsequences[i], subsequences[j])
            row[j] = score
        return i, row

    results = Parallel(n_jobs=n_jobs)(delayed(compute_row)(i) for i in tqdm(range(n), desc="Computing rows"))

    for i, row in results:
        similarity_matrix[i, i:] = row[i:]
        similarity_matrix[i:, i] = row[i:]

    return similarity_matrix

def convert_to_distance_matrix(similarity_matrix):
    """Convert similarity matrix to distance matrix using D(i, j) = 1 - normalized(S(i, j))."""
    distance_matrix = 1 - (similarity_matrix - np.min(similarity_matrix)) / (np.max(similarity_matrix) - np.min(similarity_matrix))
    np.fill_diagonal(distance_matrix, 0)  # Ensure self-distance is 0
    return distance_matrix

def cluster_subsequences(distance_matrix, num_clusters=10):
    """Cluster subsequences using hierarchical clustering."""
    condensed_distance = squareform(distance_matrix)
    linkage_matrix = linkage(condensed_distance, method='average')
    cluster_labels = fcluster(linkage_matrix, num_clusters, criterion='maxclust')
    return cluster_labels

def generate_feature_vectors(protein_subsequences, cluster_labels, all_subsequences, num_clusters):
    """Generate feature vectors for each protein based on cluster frequencies."""

    # Step 1: Map each k-mer to its cluster ID once
    kmer_to_cluster = {kmer: cluster_labels[i] for i, kmer in enumerate(all_subsequences)}

    feature_vectors = []

    for protein_kmers in tqdm(protein_subsequences, desc='generate_feature_vectors'):
        cluster_counts = np.zeros(num_clusters)

        for kmer in protein_kmers:
            cluster_id = kmer_to_cluster.get(kmer)
            if cluster_id is not None:
                cluster_counts[cluster_id - 1] += 1  # Assuming cluster IDs are 1-based

        total = cluster_counts.sum()
        if total > 0:
            feature_vector = cluster_counts / total
        else:
            feature_vector = cluster_counts  # remains zeros

        feature_vectors.append(feature_vector)

    return np.array(feature_vectors)



In [9]:
#400
def dde(sequence):
    """Computes the Dipeptide Deviation from Expected Mean (DDE) descriptor for a given protein sequence."""
    
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'  
    dipeptides = [''.join(pair) for pair in itertools.product(amino_acids, repeat=2)]
    
    aa_counts = Counter(sequence)
    L = len(sequence)
    aa_freq = {aa: aa_counts.get(aa, 0) / L for aa in amino_acids}

    dipeptide_counts = Counter([sequence[i:i+2] for i in range(L-1)])
    Dc = {dp: dipeptide_counts.get(dp, 0) / (L-1) for dp in dipeptides}

    Tm = {dp: aa_freq[dp[0]] * aa_freq[dp[1]] for dp in dipeptides}
    Tv = {dp: (Tm[dp] * (1 - Tm[dp])) / L if Tm[dp] > 0 else 0 for dp in dipeptides}

    DDE = {dp: (Dc[dp] - Tm[dp]) / (Tv[dp] ** 0.5) if Tv[dp] > 0 else 0 for dp in dipeptides}

    return DDE.values()

In [10]:
#5
hydrophobicity_Kyte_Doolittle  = {
    'A': 1.8,  'C': 2.5,  'D': -3.5, 'E': -3.5, 'F': 2.8,
    'G': -0.4, 'H': -3.2, 'I': 4.5,  'K': -3.9, 'L': 3.8,
    'M': 1.9,  'N': -3.5, 'P': -1.6, 'Q': -3.5, 'R': -4.5,
    'S': -0.8, 'T': -0.7, 'V': 4.2,  'W': -0.9, 'Y': -1.3
}

Hydrophilicity_Hopp_Woods_scale  = {
    'A': -0.5, 'C': -1.0, 'D': 3.0,  'E': 3.0,  'F': -2.5,
    'G': 0.0,  'H': -0.5, 'I': -1.8, 'K': 3.0,  'L': -1.8,
    'M': -1.3, 'N': 0.2,  'P': 0.0,  'Q': 0.2,  'R': 3.0,
    'S': 0.3,  'T': -0.4, 'V': -1.5, 'W': -3.4, 'Y': -2.3
}
Polarity_Scale = {
    'A': 8.1,  'C': 5.5,  'D': 13.0, 'E': 12.3, 'F': 5.2,
    'G': 9.0,  'H': 10.4, 'I': 5.2,  'K': 11.3, 'L': 4.9,
    'M': 5.7,  'N': 11.6, 'P': 8.0,  'Q': 10.5, 'R': 10.5,
    'S': 9.2,  'T': 8.6,  'V': 5.9,  'W': 5.4,  'Y': 6.2
}
Molecular_Weight = {
    'A': 89.09,  'C': 121.15, 'D': 133.10, 'E': 147.13, 'F': 165.19,
    'G': 75.07,  'H': 155.16, 'I': 131.17, 'K': 146.19, 'L': 131.17,
    'M': 149.21, 'N': 132.12, 'P': 115.13, 'Q': 146.15, 'R': 174.20,
    'S': 105.09, 'T': 119.12, 'V': 117.15, 'W': 204.23, 'Y': 181.19
}



def geary_autocorrelation(sequence, max_lag=5, property_dict=hydrophobicity_Kyte_Doolittle):
    
    prop_values = np.array([property_dict.get(aa, 0) for aa in sequence])  # Default 0 if AA is unknown
    N = len(prop_values)
    mean_p = np.mean(prop_values)

    geary_values = {}
    
    for d in range(1, max_lag + 1):
        numerator = np.sum((prop_values[:-d] - prop_values[d:]) ** 2)
        denominator = 2 * (N - d) * np.sum((prop_values - mean_p) ** 2)
        geary_values[f'Geary_Lag_{d}'] = (N - 1) * numerator / denominator if denominator != 0 else 0

    return geary_values.values()

In [11]:

# Read the CSV file
df = pd.read_csv("/kaggle/input/virus-drug/virus_drug_interactions.csv")
df = df.drop(df.columns[0], axis=1)
# Display the first few rows
print(df.head())

                                    Protein_Sequence  \
0  PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...   
1  MTMDEQQSQAVAPVYVGGFLARYDQSPDEAELLLPRDVVEHWLHAQ...   
2  PQVTLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...   
3  PQVTLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...   
4  PQVTLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...   

                                              SMILES     pIC50  
0                       S=C(NCN1CCOCC1)Nc1ccc(Br)cn1  5.000000  
1                  CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C  4.000000  
2  CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...  7.522879  
3  CCC(C)[C@H](NC(=O)[C@@H]1CCCN1[P@@](=O)(OC)[C@...  7.031517  
4  COC(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)C(O)CN(Cc...  7.376751  


In [12]:
# Example Data
protein_sequences = df['Protein_Sequence']
k = 5
num_clusters = 10

# Extract subsequences (k-mers)
all_subsequences, protein_subsequences = extract_subsequences(protein_sequences, k)
# Compute similarity and distance matrices
all_subsequences=list(all_subsequences)
similarity_matrix = compute_similarity_matrix(all_subsequences)
distance_matrix = convert_to_distance_matrix(similarity_matrix)

# Cluster the subsequences
subsequence_cluster_labels = cluster_subsequences(distance_matrix, num_clusters)
# Generate feature vectors for each protein
feature_vectors = generate_feature_vectors(protein_subsequences, subsequence_cluster_labels, all_subsequences, num_clusters)

print(len(feature_vectors),len(feature_vectors[0]))


generate_feature_vectors: 100%|██████████| 19451/19451 [00:05<00:00, 3883.13it/s]


19451 10


In [22]:
def one_hot_encode(value, valid_values):
    if value not in valid_values:
        value = valid_values[-1]
    return [value == item for item in valid_values]


def get_atom_features(atom):
    atom_symbols = [
        'C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca',
        'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn',
        'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au',
        'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr', 'Pt', 'Hg', 'Pb', 'X'
    ]
    degrees = list(range(11))
    hydrogen_counts = list(range(11))
    valences = list(range(11))

    features = (
        one_hot_encode(atom.GetSymbol(), atom_symbols) +
        one_hot_encode(atom.GetDegree(), degrees) +
        one_hot_encode(atom.GetTotalNumHs(), hydrogen_counts) +
        one_hot_encode(atom.GetImplicitValence(), valences) +
        [atom.GetIsAromatic()]
    )

    return np.array(features)


In [19]:
def smile_graph(smile):
    nodes=[]
    edges=[]
    edges_type=[]
    mol = Chem.MolFromSmiles(smile)
    mol_size = mol.GetNumAtoms()
    for atom in mol.GetAtoms():
        nodes.append(get_atom_features(atom))
    
    for bond in mol.GetBonds():
        start = bond.GetBeginAtomIdx()
        end = bond.GetEndAtomIdx()
        bond_type = bond.GetBondTypeAsDouble()

        # Since molecular graphs are undirected, add both directions
        edges.append([start, end])
        edges.append([end, start])

        edges_type.append(bond_type)
        edges_type.append(bond_type)
        

    return mol_size,nodes,edges,edges_type

    # for atom in mol.GetAtoms():
    #     print(atom.GetIdx(), atom.GetSymbol())
    # return  Draw.MolToImage(mol)
    
   
    

In [23]:
protein_sequences = df['Protein_Sequence']
drugs = df['SMILES']
pIC50=df['pIC50']
protein_features=[]
drug_graphs=[]
for i,protein in tqdm(enumerate(protein_sequences)):
    vec=[]
    vec.extend(apaac(protein))
    vec.extend(ctdd(protein))
    vec.extend(ctriad(protein))
    vec.extend(dde(protein))
    vec.extend(geary_autocorrelation(protein))
    vec.extend(feature_vectors[i])

    protein_features.append(vec)
    graph=smile_graph(drugs[i])
    drug_graphs.append(graph)
print(len(protein_features),len(protein_features[0]))
print(len(drug_graphs),len(drug_graphs[0]))



19451it [02:29, 130.28it/s]


19451 803
19451 4


In [25]:
protein_features=np.array(protein_features)
pIC50=np.array(pIC50)

np.save("protein_features.npy", protein_features)
np.save("pIC50.npy", pIC50)


In [26]:
import pickle

with open("drug_graphs.pkl", "wb") as f:
    pickle.dump(drug_graphs, f)

In [27]:
with open("/kaggle/working/drug_graphs.pkl", "rb") as f:
    drug_graphs = pickle.load(f)



18 [array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False]), array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, 

In [ ]:
def drug_graph_to_data(drug_graph):
    mol_size, nodes, edges, edges_type = drug_graph
    x = torch.tensor(nodes, dtype=torch.float)  # [num_nodes, node_features]
    
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()  # [2, num_edges]
    edge_attr = torch.tensor(edges_type, dtype=torch.float).unsqueeze(1)  # [num_edges, 1]
    
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    return data

In [ ]:
class DrugProteinDataset(torch.utils.data.Dataset):
    def __init__(self, protein_features, drug_graphs, pIC50_values):
        self.protein_features = protein_features
        self.drug_graphs = drug_graphs
        self.pIC50_values = pIC50_values
    
    def __len__(self):
        return len(self.pIC50_values)
    
    def __getitem__(self, idx):
        protein_feature = torch.tensor(self.protein_features[idx], dtype=torch.float)
        drug_graph = drug_graph_to_data(self.drug_graphs[idx])
        pIC50_value = torch.tensor(self.pIC50_values[idx], dtype=torch.float)
        return protein_feature, drug_graph, pIC50_value

def custom_collate(batch):
    protein_feats = torch.stack([item[0] for item in batch])  # [batch_size, protein_feature_dim]
    drug_graphs = [item[1] for item in batch]                 # List of PyG Data objects
    labels = torch.stack([item[2] for item in batch])         # [batch_size]

    batch_drug_graphs = Batch.from_data_list(drug_graphs)     # Combine graphs into a single batched graph

    return protein_feats, batch_drug_graphs, labels


In [ ]:
class DrugTargetGNN(nn.Module):
    def __init__(self, node_feature_dim=-1, protein_feature_dim=-1, hidden_dim=64):
        super().__init__()
        # GNN layers for drug graph
        self.conv1 = GCNConv(node_feature_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        
        # MLP for protein features
        self.protein_mlp = nn.Sequential(
            nn.Linear(protein_feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Final layers for combined features
        self.final_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)  # regression output for pIC50
        )
        
    def forward(self, protein_feat, drug_graph):
        # GNN on drug graph
        x, edge_index = drug_graph.x, drug_graph.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, drug_graph.batch)  # [batch_size, hidden_dim]
        
        # Protein feature embedding
        p = self.protein_mlp(protein_feat)  # [batch_size, hidden_dim]
        
        # Combine embeddings
        combined = torch.cat([x, p], dim=1)
        out = self.final_mlp(combined)
        return out.squeeze()  # [batch_size]

In [ ]:

dataset = DrugProteinDataset(protein_features, drug_graphs, pIC50)
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size  # ensure no rounding issues

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)  # for reproducibility
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, collate_fn=custom_collate)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DrugTargetGNN(protein_feature_dim=len(protein_features[0])).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()  # for regression

model.train()
for epoch in range(20):
    total_loss = 0
    for protein_feat, drug_graph, values in train_loader:
        protein_feat = protein_feat.to(device)
        drug_graph = drug_graph.to(device)
        values = values.to(device)

        optimizer.zero_grad()
        outputs = model(protein_feat, drug_graph)
        loss = criterion(outputs, values)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

In [ ]:

def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for protein_feats, drug_graphs, values in dataloader:
            protein_feats = protein_feats.to(device)
            drug_graphs = drug_graphs.to(device)
            labels = labels.to(device)

            outputs = model(protein_feats, drug_graphs)
            all_preds.append(outputs.cpu())
            all_labels.append(labels.cpu())

    preds = torch.cat(all_preds).numpy()
    labels = torch.cat(all_labels).numpy()

    mse = mean_squared_error(values, preds)
    rmse = mse ** 0.5
    pearson_corr, _ = pearsonr(values, preds)

    return {
        "MSE": mse,
        "RMSE": rmse,
        "Pearson": pearson_corr
    }


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

val_metrics = evaluate(model, val_loader, device)
print("Validation Metrics:", val_metrics)